# Problem 1: The Mandelbrot Fractal

This is a fun problem, for several reasons:

* It produces pretty pictures
* There are lots of variations to play with
* The algorithm can exit early, making it not trivial to vectorize

Let's import some libraries. Note, to automatically see plots, sometimes you may have to do:
```python
%matplotlib inline
```

(or `notebook`, `widget`)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Extra performance libraries
import numexpr
import numba

You can generate a Mandelbrot fractal by applying the transform:

$$
z_{n+1}=z_{n}^{2}+c
$$

repeatedly to a regular matrix of complex numbers $c$, and recording the iteration number where the value $|z|$ surpassed some bound, usually 2. You start at $z_0 = c$.



Let's set up some inital parameters and a helper matrix:

In [ ]:
maxiterations = 50

# 300 x 400 matrix of complex numbers from [-1.5j, 1.5j] x [-2, 2]
c = np.sum(np.ogrid[-1.5j:1.5j:300j, -2:2:400j])

Let's make sure we have the correct `c`:

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(6,3))
axs[0].pcolormesh(c.real, c.imag, c.real)
axs[1].pcolormesh(c.real, c.imag, c.imag);

Okay, let's make the fractal as simply as possible in numpy:

In [ ]:
fractal = np.zeros_like(c, dtype=np.int32)
z = c.copy()

for i in range(1, maxiterations+1):
    z = z**2 + c                 # Compute z
    diverge = abs(z) > 2         # Divergence criteria
    
    z[diverge] = 2               # Keep number size small
    fractal[~diverge] = i        # Fill in non-diverged iteration number

In [ ]:
fig, ax = plt.subplots(figsize=(4,3))
ax.pcolormesh(c.real, c.imag, fractal)

and in pure python (for use later):

In [ ]:
fractal = np.zeros_like(c, dtype=np.int32)

for yi in range(c.shape[0]):
    for xi in range(c.shape[1]):
        z = cxy = c[yi, xi]
        for i in range(1, maxiterations+1):
            z = z**2 + cxy
            if abs(z) > 2:
                break
            else:
                fractal[yi, xi] = i

In [ ]:
fig, ax = plt.subplots(figsize=(4,3))
ax.pcolormesh(c.real, c.imag, fractal)

In [ ]:
def fractal_numpy(c, maxiterations):
    f = np.zeros_like(c, dtype=np.int32)
    z = c.copy()

    for i in range(1, maxiterations+1):
        z = z**2 + c                 # Compute z
        diverge = abs(z**2)  > 2**2  # Divergence criteria

        z[diverge] = 2               # Keep number size small
        f[~diverge] = i              # Fill in non-diverged iteration number
        
    return f

In [ ]:
def on_each_python(cxy, maxiterations):
    z = cxy
    for i in range(1, maxiterations+1):
        z = z**2 + cxy
        if abs(z) > 2:
            return i
    return i

def fractal_python(c, maxiterations=50):
    fractal = np.zeros_like(c, dtype=np.int32)

    for yi in range(c.shape[0]):
        for xi in range(c.shape[1]):
            fractal[yi, xi] = on_each_python(c[yi, xi], maxiterations)
    
    return fractal

def fractal_pure(c, maxiterations):
    fractal = np.zeros_like(c, dtype=np.int32)

    for yi in range(c.shape[0]):
        for xi in range(c.shape[1]):
            z = cxy = c[yi, xi]
            for i in range(1, maxiterations+1):
                z = z**2 + cxy
                if abs(z) > 2:
                    break
                else:
                    fractal[yi, xi] = i
    return fractal

In [ ]:
%%timeit
fractal_python(c, maxiterations)

In [ ]:
%%timeit
fractal_numpy(c, maxiterations)

In [ ]:
%%timeit
fractal_pure(c, maxiterations)

In [ ]:
fractal = fractal_pure(c, maxiterations)
fig, ax = plt.subplots(figsize=(4,3))
ax.pcolormesh(c.real, c.imag, fractal)

### Profiling

Never optimize until you have profiled! If code becomes uglier/harder to maintain, you *must* have a solid reason for doing so.

Let's look at the `line_profiler` package, which has fairly nice IPython magic support. First let's load the magic:

In [ ]:
%load_ext line_profiler

Now, we run the magic with `-f function_to_profile` and the command to profile. Only the lines of the function we specify will show up:

In [ ]:
%lprun -f fractal_numpy fractal_numpy(c, maxiterations)

If you don't have external packages available, the built-in `cProfile` is also usable, though not as pretty.

> #### Note
>
> Most standard library modules with names like `something, cSomething` were merged in Python 3, with the faster compiled implementation being selected automatically. This one was not, since `cProfile` and `profile` are not quite identical. `profile` is much slower.

In [ ]:
import cProfile

In [ ]:
cProfile.run('fractal_numpy(c, maxiterations)', sort=2)

(Note: Numba takes full advantage of the instruction set on your system, since it does not expect to be compiled and run on a different machine; thus often Numba will be faster than normally compiled code).

### Project: accelerate

Try some of the following:

* Use `numexpr` to replace parts of the above calculation. Why is this not very effective?
* Try reducing the number of memory allocations by using numpy
* Try accelerating using `@numba.njit`
* Try accelerating using `@numba.vectorize`

Further reading:

* [Christoph Deil's Numba talk](https://christophdeil.com/download/2019-07-11_Christoph_Deil_Numba.pdf)
* [CompClass](https://github.com/henryiii/compclass): Several days visited this, including week 12
* Any of Jim's classes (see intro talk)
* The distributed lesson will revisit fractals